### Run the query in Wikidata Query Service
* https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/110

In [1]:
import time
from datetime import datetime

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-07-17 22:11:35


In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """#title: Miss Instagram Location connected to SAT
SELECT DISTINCT ?id ?geo ?idLabel ?img ?SATLabel ?Instagram ?Facebook ?InstagramL ?Googlemap WHERE {
  ?id wdt:P6104 wd:Q134294510;
       wdt:P625 ?geo.

  ?id wdt:P2789 ?SAT.
  ?SAT wdt:P361 wd:Q131318799.

  OPTIONAL {?id wdt:P2003 ?InstaU}
  minus {?id wdt:P4173 ?InstaL}

  BIND(URI(CONCAT("https://www.instagram.com/explore/locations/", ?InstaL)) AS ?InstagramL)

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,nn". }
} order by ?SATLabel 
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)


Manually search using the Label + coordinates in Google:
* site:instagram.com/explore/locations/ "Mock Location 2"

### Next try

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON

def fetch_trail_items():
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""
      SELECT ?item ?coord ?itemLabel WHERE {
        ?item wdt:P6104 wd:Q134294510;
              wdt:P625 ?coord.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
      }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return [{
      "qid": r["item"]["value"].split("/")[-1],
      "label": r["itemLabel"]["value"],
      "coord": r["coord"]["value"].replace("Point(", "").rstrip(")").split()
    } for r in results["results"]["bindings"]]


In [4]:
from instagrapi import Client
import time
import random
from SPARQLWrapper import SPARQLWrapper, JSON

cl = Client()
cl.login("salgo60@msn.com", "salgo")

def fetch_trail_items():
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""
      SELECT ?item ?coord ?itemLabel WHERE {
        ?item wdt:P6104 wd:Q134294510;
              wdt:P625 ?coord.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
      }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return [{
      "qid": r["item"]["value"].split("/")[-1],
      "label": r["itemLabel"]["value"],
      "coord": r["coord"]["value"].replace("Point(", "").rstrip(")").split()
    } for r in results["results"]["bindings"]]

def find_instagram_location(lat, lng, retries=1):
    try:
        locations = cl.location_search(lat=float(lat), lng=float(lng))
        return locations[0] if locations else None
    except Exception as e:
        print(f"⚠️ Error: {e}. Retrying in 60 seconds...")
        if retries > 0:
            time.sleep(60)
            return find_instagram_location(lat, lng, retries - 1)
        return None

items = fetch_trail_items()
qs_lines = []
request_count = 0

for i, item in enumerate(items):
    lat, lng = item["coord"][1], item["coord"][0]
    print(f"\n🔎 [{i+1}/{len(items)}] Processing: {item['label']} at ({lat}, {lng})")

    insta = find_instagram_location(lat, lng)
    request_count += 1

    if insta:
        qs_line = f"{item['qid']}\tP4173\t{insta.pk}"
        print(f"✅ Found Instagram ID: {insta.pk} → {qs_line}")
        qs_lines.append(qs_line)
    else:
        print("❌ No Instagram location found.")

    sleep_time = random.uniform(10.5, 1000.5)
    print(f"⏳ Sleeping for {sleep_time:.1f} seconds... | Total requests so far: {request_count}")
    time.sleep(sleep_time)

# Save to QuickStatements file
with open("add_instagram_locations.qs", "w") as f:
    f.write("\n".join(qs_lines))

print(f"\n✅ Done! Processed {len(items)} items, {len(qs_lines)} Instagram IDs found.")


ChallengeRequired: challenge_required

In [ ]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))